#1. Introduccion

El Ejercicio 2 del TP3 tiene como objetivo obtener los  hospitales más cercanos a la ubicación del usuario y calcular las distancias a los mismos. Los hospitales se obtienen mediante la API de "here", y la ubicación del usuario (en este programa será la de la UNLaM) con el formato de latitud y longitud. 

Este proyecto surge como una mejora a la aplicacion Covid Helper, en la cual se hacia el calculo de las distancias  de los hospitales de CABA a la ubicación del usuario (que se obtiene mediante el gps del celular). Se espera mejorar el rendimiento para el procesamiento de grandes volumenes de informacion de los hospitales.

Realizaremos la ejecución del algoritmo de manera secuencial, haciendo uso de la CPU; y luego haciendo uso de la GPU, con CUDA, realizando una ejecución paralela mediante threads.

#2. Armado del ambiente


In [ ]:
#--------------------------------------INSTALACION DE BIBLIOTECA 'REQUESTS'--------------------------------------#
!pip install requests

In [ ]:
#--------------------------------------INSTALACION DE BIBLIOTECA 'PYCUDA'--------------------------------------#
!pip install pycuda

     |████████████████████████████████| 1.7MB 9.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627583 sha256=d02c3dc697bab7dd4ba532ad20d67598b3bda5c41878eb85c5ca9b98c74e5568
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.7-py2.py3-none-any.whl size=60644 sha256=cf03b242cc26e18b74d967be731a6c5db2dfbea0fdb7f75c5bd73b135771c075
  Stored in directory: /root/.cache/pip/wheels/a0/b5/e5/e65d25997fd77729b9aa214645add18688483e48bbcbab6ffc
Successfully built pytools


#3. Desarrollo

#CPU

In [ ]:
#--------------------------------------BIBLIOTECAS--------------------------------------#

from datetime import datetime

#Obtencion del tiempo inicial
tiempo_total = datetime.now()

import numpy as np
from decimal import Decimal
import requests
import sys
from math import radians, cos, sin, asin, sqrt

#----------------------------------------FUNCION----------------------------------------#
#Declaracion de funcion que realiza el pasaje del tiempo obtenido mediante datetime.now(), a milisegundos.
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#Funcion que calcula la distancia entre dos puntos mediante la utilizacion de latitudes y longitudes
def distancias_hospitales(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6371* c
    return km
#---------------------------------------CODIGO--------------------------------------------#

#URL de la API para acceder hospitales
URL       = "https://discover.search.hereapi.com/v1/discover"

#Clave para acceder a la API
api_key   = 'tz3In-98cqt460SzMIiDpLpRxKsLYSC1Sb_rNVL_3Tk'

#Parametro para solicitar hospitales en la API
query     = 'hospitals'

#Cantidad de hospitales a solicitar
limit     = 100

#Latitud y Longitud del usuario, en este caso es la de la UNLaM
latitude  = -34.6807496
longitude = -58.5070069

#Formato del request para la API 
PARAMS    = {
            'apikey':api_key,
            'q':query,
            'limit': limit,
            'at':'{},{}'.format(latitude,longitude)
            }

#Conexion con la API
request = requests.get(url = URL, params = PARAMS) 

#Obtencion de la respuesta de la API como JSON
data = request.json()

#Declaracion de listas con los datos a utilizar
hospitales_nombre     = []
hospitales_ciudad     = []
hospitales_latitud    = []
hospitales_longitud   = []

#Almacenamiento de la informacion de los hospitales del JSON a las listas.
for i in range(limit):
  nombre = data['items'][i]['title'] #Obtencion del nombre del hospital
  ciudad =  data['items'][i]['address']['city'] #Obtencion de la ciudad en la que encuentra el hospital
  latitud = data['items'][i]['position']['lat'] #Obtencion de la latitud del hospital
  longitud = data['items'][i]['position']['lng'] #Obtencion de la longitud del hospital
  hospitales_nombre.append(nombre) 
  hospitales_ciudad.append(ciudad) 
  hospitales_latitud.append(latitud) 
  hospitales_longitud.append(longitud) 

cant_elementos_test = limit*100

for i in range (cant_elementos_test):
  hospitales_nombre.append(nombre) 
  hospitales_ciudad.append(ciudad) 
  hospitales_latitud.append(latitud) 
  hospitales_longitud.append(longitud) 

#Generacion de array de latitud y longitud en base a las listas de latitudes y longitudes de los hospitales
array_latitud = np.array(hospitales_latitud)
array_longitud = np.array(hospitales_longitud)

#Conversion de los datos del array a tipo de dato float
array_latitud = array_latitud.astype(np.float32())
array_longitud = array_longitud.astype(np.float32())

#Generacion de un vector para las distancias
array_distancia = np.empty_like(array_latitud)

#Obtencion del tiempo previo a la ejecucion en GPU
tiempo_calculo_cpu = datetime.now()

#Ejecucion de los calculos de las distancias.
for i in range(cant_elementos_test):
  array_distancia[i] = distancias_hospitales(longitude, latitude, array_longitud[i], array_latitud[i])

#Calculo del tiempo total de procesamiento de la imagen en GPU
tiempo_calculo_cpu = datetime.now() - tiempo_calculo_cpu

#Pasaje del resultado, de lista a array.
distancias = array_distancia.tolist()

#Muestra de todos los hospitales, ordenados por distancia.
#Se muestra:
# -Nombre
# -Ciudad
# -Latitud
# -Longitud
# -Distancia
print("########################################################################################### HOSPITALES ###########################################################################################")
print()
for i in range(cant_elementos_test):
    print(hospitales_nombre[i].rjust(36), end='')
    print(hospitales_ciudad[i].rjust(36), end='')
    print(str(hospitales_latitud[i]).rjust(36), end='')
    print(str(hospitales_longitud[i]).rjust(36), end='')
    print(str(distancias[i]).rjust(36), end='')
    print()

#Calculo del tiempo total de ejecucion del programa
tiempo_total = datetime.now() - tiempo_total

print()
print()
print("########################################################################################### TIEMPOS ###########################################################################################")
print()
print("Tiempo total de ejecucion: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo de procesamiento en CPU: ", tiempo_en_ms( tiempo_calculo_cpu   ), "[ms]" )
print()
print("###############################################################################################################################################################################################")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                  17.577131271362305
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                  17.577131271362305
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                  17.577131271362305
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                  17.577131271362305
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                  17.577131271362305
          Hospital Dr. Eduar

#GPU

In [ ]:
#--------------------------------------BIBLIOTECAS--------------------------------------#

from datetime import datetime

#Obtencion del tiempo inicial
tiempo_total = datetime.now()

import numpy as np
from decimal import Decimal
import requests
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import sys

#----------------------------------------FUNCION----------------------------------------#

#Declaracion de funcion que realiza el pasaje del tiempo obtenido mediante datetime.now(), a milisegundos.
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#---------------------------------------CODIGO--------------------------------------------#

#URL de la API para acceder hospitales
URL       = "https://discover.search.hereapi.com/v1/discover"

#Clave para acceder a la API
api_key   = 'tz3In-98cqt460SzMIiDpLpRxKsLYSC1Sb_rNVL_3Tk'

#Parametro para solicitar hospitales en la API
query     = 'hospitals'

#Cantidad de hospitales a solicitar
limit     = 100

#Latitud y Longitud del usuario
latitude  = -34.6807496
longitude = -58.5070069

#Formato del request para la API 
PARAMS    = {
            'apikey':api_key,
            'q':query,
            'limit': limit,
            'at':'{},{}'.format(latitude,longitude)
            }

#Conexion con la API
request = requests.get(url = URL, params = PARAMS) 

#Obtencion de la respuesta de la API como JSON
data = request.json()

#Declaracion de listas con los datos a utilizar
hospitales_nombre     = []
hospitales_ciudad     = []
hospitales_latitud    = []
hospitales_longitud   = []

#Almacenamiento de la informacion de los hospitales del JSON a las listas.
for i in range(limit):
  nombre = data['items'][i]['title'] #Obtencion del nombre del hospital
  ciudad =  data['items'][i]['address']['city'] #Obtencion de la ciudad en la que encuentra el hospital
  latitud = data['items'][i]['position']['lat'] #Obtencion de la latitud del hospital
  longitud = data['items'][i]['position']['lng'] #Obtencion de la longitud del hospital
  hospitales_nombre.append(nombre) 
  hospitales_ciudad.append(ciudad) 
  hospitales_latitud.append(latitud) 
  hospitales_longitud.append(longitud) 

cant_elementos_test = limit*100

for i in range (cant_elementos_test):
  hospitales_nombre.append(nombre) 
  hospitales_ciudad.append(ciudad) 
  hospitales_latitud.append(latitud) 
  hospitales_longitud.append(longitud) 

#Generacion de array de latitud y longitud en base a las listas de latitudes y longitudes de los hospitales
array_latitud = np.array(hospitales_latitud)
array_longitud = np.array(hospitales_longitud)

#Conversion de los datos del array a tipo de dato float
array_latitud = array_latitud.astype(np.float32())
array_longitud = array_longitud.astype(np.float32())

#Generacion de un vector para las distancias
array_distancia = np.empty_like(array_latitud)

#Reserva de memoria para los arrays en GPU
array_latitud_gpu = cuda.mem_alloc( array_latitud.nbytes )
array_longitud_gpu = cuda.mem_alloc( array_longitud.nbytes )
array_distancia_gpu = cuda.mem_alloc( array_distancia.nbytes )

#Copia de arrays de cpu a gpu
cuda.memcpy_htod( array_latitud_gpu, array_latitud )
cuda.memcpy_htod( array_longitud_gpu, array_longitud )
cuda.memcpy_htod( array_distancia_gpu, array_distancia )

#Declaracion y desarrollo de las funciones en cuda.
#Se desarrolla:
# -Pasaje de grados a radianes
# -Calculo de la distancia entre dos coordenadas especificas
# -Calculo de  la distancia para cada hospital recibido desde la API
funcion_cuda = SourceModule(""" 
#include <stdio.h>
#include <math.h>
#define PI 3.14159265358979323846

__device__ float grados_a_radianes(float grados)
{
    return grados * PI / 180;
}


__device__ float calcular_distancia_dos_coordenadas(float lato, float lono, float latd, float lond)
{
    // Convertir todas las coordenadas a radianes
    float lat1 = grados_a_radianes(lato);
    float lon1 = grados_a_radianes(lono);
    float lat2 = grados_a_radianes(latd);
    float lon2 = grados_a_radianes(lond);
    // Aplicar fórmula
    float RADIO_TIERRA_EN_KILOMETROS = 6371;
    float diferencia_longitudes = (lon2 - lon1);
    float diferencia_latitudes = (lat2 - lat1);
    float a = pow(sin(diferencia_latitudes / 2.0), 2) + cos(lat1) * cos(lat2) * pow(sin(diferencia_longitudes / 2.0), 2);
    float c = 2 * atan2(sqrt(a), sqrt(1 - a));
    float d = RADIO_TIERRA_EN_KILOMETROS * c;
    return d;
}


__global__ void distancias_hospitales( int cant, float latitud, float longitud, float *hsp_latitud, float *hsp_longitud, float *hsp_distancia )
{
  int x = threadIdx.x + blockIdx.x*blockDim.x;

  if(x <= cant)
  {
    
    hsp_distancia[x] = calcular_distancia_dos_coordenadas(latitud, longitud, hsp_latitud[x], hsp_longitud[x]);
    printf("Origen: %f %f, destino: %f %f, distancia: %f",latitud, longitud, hsp_latitud[x], hsp_longitud[x],hsp_distancia[x] );
  }
}
""")

#Obtencion de la funcion para obtener las distancias de los hospitales en cuda.
funcion_gpu = funcion_cuda.get_function("distancias_hospitales")

#Calculo de cantidad de hilos. Limit es la cantidad de hospitales a solicitar a la API.
dim_hilo = limit
dim_bloque = np.int( (cant_elementos_test+dim_hilo-1) / dim_hilo )

#Obtencion del tiempo previo a la ejecucion en GPU
tiempo_calculo_gpu = datetime.now()

#Ejecucion en GPU
funcion_gpu( np.int32(cant_elementos_test), np.float32(latitude), np.float32(longitude), array_latitud_gpu, array_longitud_gpu, array_distancia_gpu,   block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1))

#Calculo del tiempo total de procesamiento de la imagen en GPU
tiempo_calculo_gpu = datetime.now() - tiempo_calculo_gpu

#Copia del resultado de la ejecucion en GPU a CPU.
cuda.memcpy_dtoh( array_distancia, array_distancia_gpu )

#Pasaje del resultado, de lista a array.
distancias = array_distancia.tolist()

#Muestra de todos los hospitales, ordenados por distancia.
#Se muestra:
# -Nombre
# -Ciudad
# -Latitud
# -Longitud
# -Distancia
print("########################################################################################### HOSPITALES ###########################################################################################")
print()
for i in range(cant_elementos_test):
    print(hospitales_nombre[i].rjust(36), end='')
    print(hospitales_ciudad[i].rjust(36), end='')
    print(str(hospitales_latitud[i]).rjust(36), end='')
    print(str(hospitales_longitud[i]).rjust(36), end='')
    print(str(distancias[i]).rjust(36), end='')
    print()

#Calculo del tiempo total de ejecucion del programa
tiempo_total = datetime.now() - tiempo_total

print()
print()
print("########################################################################################### TIEMPOS ###########################################################################################")
print()
print("Tiempo total de ejecucion: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo de procesamiento en GPU: ", tiempo_en_ms( tiempo_calculo_gpu   ), "[ms]" )
print()
print("###############################################################################################################################################################################################")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                   17.57703399658203
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                   17.57703399658203
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                   17.57703399658203
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                   17.57703399658203
          Hospital Dr. Eduardo Wilde                               Wilde                           -34.69593                           -58.31565                   17.57703399658203
          Hospital Dr. Eduar

#4. Conclusiones

El algoritmo esta diseñado para ejecutar grandes volumenes de informacion de hospitales (actualmente la API brinda los 100 más cercanos) y hacer los calculos de las distancias de los mismos con la ubicacion del usuario. Originalmente el desarrollo se hizo para que ejecutara en CPU, pero en esta mejora se realiza para que pueda ejecutar en GPU. Además, permite obtener más hospitales, no solamente los de CABA.

Al hacer los calculos de las distancias en GPU, se podrá obtener los resultados finales mucho más rapido que en CPU que ejecuta de manera secuencial. Las mejoras en los tiempos se podrán ver cuando se realicen los calculos con grandes de volumenes informacion, si se utiliza poca informacion la CPU ejecutará más rapido.

Para testear el rendimiento, hicimos pruebas con 10  y con 10100 elementos.


> - Para 10100 elementos pudimos observar que en CPU los calculos de las distancias tomó en promedio 17.688 ms en CPU y 0.323 ms en GPU. Podemos ver que hacer procesamiento con GPU resulta en promedio 54000 veces más rapido que con CPU.



> - Para 10 elementos pudimos observar que en CPU los calculos de las distancias tomó en promedio 0.097 ms en CPU y 0.297 ms en GPU. Podemos ver que hacer procesamiento con GPU resulta en promedio 3 veces más rapido que con CPU.


Como conclusion podemos observar que realizar procesamiento con GPU resulta mucho mas veloz que con CPU cuando se trata de grandes cantidades de informracion a procesar. Si se trata de poca informacion a procesar, no es conveniente utilizar GPU ya que considerando el tiempo de las reservas de memoria y el copiado de los arrays en GPU, demorará más en ejecutar.


#5. Bibliografía

[1] Cuda: [Documentación Oficial Nvidia](https://docs.nvidia.com/cuda/)

[2] Here: [Documentación Here](https://developer.here.com/documentation#services)

[3] Covid Helper: [Covid Helper](https://github.com/SantiagoAlbarracin/Covid-Helper)

[4] Python basico: [Python Basico-SOA UNLaM](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)